# Random predictions to SageMaker endpoint

**Note:** Please set kernel to `Python 3 (Data Science)`

---

Before running this notebook, ensure that notebook 03 ran successfully and the updated files were committed to CodeCommit. Also, ensure the SageMaker Pipeline execution is successful and the staging endpoint shows `InService`. You can check the endpoint status in the SageMaker project page, under the Endpoints tab. 

In the screenshot below, my project name is `BankDM` and the endpoint is `BankDM-staging`.

![endpoint](img/check-endpoint1.png)

## Introduction

Now that the model is trained and deployed, the next step is to use the model to do predictions. You will first connect to RedShift and retrieve the data. Before doing predictions, you need to do preprocessing of the data to ensure it is of the same format as the training data. 

### Variables
Variable name for secret in Secret Manager and SageMaker endpoint. RedShift, Athena and Glue information are stored in the secret.

In [1]:
secret_name='bankdm_redshift_login' 

# The endpoint name below is the default based on the SageMaker project name
# endpoint name is case sensitive
endpoint_name = 'BankDM-staging'

### Install and import libraries

In [2]:
!pip install -q SQLAlchemy==1.3.13
!pip install psycopg2-binary pyathena
!pip install -U pip
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from pyathena import connect
from botocore.exceptions import ClientError
import numpy as np
import pandas as pd
import time
import json
import boto3
import sagemaker
from sagemaker.serializers import CSVSerializer
import random
import math

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils 

### Create client session


In [3]:
# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.client('s3')
redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')

session = boto3.session.Session()
region = session.region_name

### Get credentials & connection information from Secret Manager

In [4]:
try:
    get_secret_value_response = secretsmanager.get_secret_value(
            SecretId=secret_name
        )
    secret_arn=get_secret_value_response['ARN']

except ClientError as e:
    print("Error retrieving secret. Error: " + e.response['Error']['Message'])
    
else:
    # Depending on whether the secret is a string or binary, one of these fields will be populated.
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
    else:
        secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
secret_json = json.loads(secret)
master_user_name = secret_json['username']
master_user_pw = secret_json['password']
redshift_port = secret_json['port']
redshift_cluster_identifier = secret_json['dbClusterIdentifier']
redshift_endpoint_address = secret_json['host']

database_name_redshift = secret_json['database_name_redshift']
database_name_glue = secret_json['database_name_glue']

schema_redshift = secret_json['schema_redshift']
schema_athena = secret_json['schema_athena']

table_name_glue = secret_json['table_name_glue']
table_name_redshift = secret_json['table_name_redshift']

# print(master_user_name)

## RedShift

### Connect to RedShift

In [5]:
response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(master_user_name, master_user_pw, redshift_endpoint_address, redshift_port, database_name_redshift))
session = sessionmaker()
session.configure(bind=engine)


### Get data from RedShift and shuffle it

In [6]:
statement = f"""
select *  
    FROM {schema_redshift}.{table_name_redshift} order by random()
"""

# print(statement)

data = pd.read_sql_query(statement, engine)
data.head(5)

,age,job,marital,education,defaulted,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,40,technician,married,professional.course,no,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.0,93.0,-36.0,5.0,None,no
1,31,services,single,high.school,no,no,no,telephone,may,fri,...,3,999,0,nonexistent,1.0,94.0,-36.0,5.0,None,no
2,35,blue-collar,married,basic.9y,unknown,no,no,cellular,jul,fri,...,3,999,0,nonexistent,1.0,94.0,-43.0,5.0,None,no
3,41,blue-collar,married,basic.4y,unknown,yes,no,cellular,apr,fri,...,1,999,1,failure,-2.0,93.0,-47.0,1.0,None,no
4,48,technician,married,professional.course,no,yes,no,cellular,sep,mon,...,1,999,0,nonexistent,-3.0,92.0,-30.0,1.0,None,no


### Pre-processing of data

In [7]:
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed
model_data = pd.get_dummies(data)                                                                  # Convert categorical variables to sets of indicators

model_data = model_data.drop(['duration', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m'], axis=1)
df = pd.concat([model_data['y_yes'], model_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
df.columns

Index(['y_yes', 'age', 'campaign', 'pdays', 'previous', 'no_previous_contact',
       'not_working', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'defaulted_no', 'defaulted_unknown',
       'defaulted_yes', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_unknown', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'month_apr', 'month_aug', 'month_dec', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'day_of_week_fri', 'd

Extract out the target column to another dataframe and drop the column in the existing dataframe.

In [8]:
df_y_yes = df['y_yes']
df = df.drop(['y_yes'], axis=1)
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'no_previous_contact',
       'not_working', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'defaulted_no', 'defaulted_unknown',
       'defaulted_yes', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_unknown', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'month_apr', 'month_aug', 'month_dec', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'day_of_week_fri', 'day_of_wee

Check the target value dataframe is correct

In [9]:
df_y_yes.head()

0    0
1    0
2    0
3    0
4    0
Name: y_yes, dtype: uint8

Check the original dataframe

In [10]:
arr_test =df[df.columns[:]].values
arr_test

array([[ 40,   1, 999, ...,   0,   1,   0],
       [ 31,   3, 999, ...,   0,   1,   0],
       [ 35,   3, 999, ...,   0,   1,   0],
       ...,
       [ 51,   3, 999, ...,   0,   1,   0],
       [ 40,   3, 999, ...,   0,   1,   0],
       [ 34,   1, 999, ...,   1,   0,   0]])

In [11]:
predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)
predictor.serializer = CSVSerializer()

Get a prediction

In [12]:
# Randomly select a user and predicts
i = random.randint(1, len(arr_test)) 
print(i)
pred = predictor.predict(arr_test[:][i])
pred

36848


b'0.030248522758483887'

Check the actual result

In [13]:
# actual value
df_y_yes[i]

0

Compare if the pred and actual result are similar with a tolerance of 0.1

In [14]:
math.isclose(float(pred), df_y_yes[i], abs_tol=0.1)

True

---

## Next steps

Now that prediction using a SageMaker endpoint works, the next step is to use RedShift ML to do prediction as well.